In [43]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from minineedle import needle, smith, core

In [44]:
#strict consistency
path_no_context_strict_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/ChatGPT/strict_consistency/'
path_no_context_strict_consistency_googlebard = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/GoogleBard/strict_consistency/'
path_no_context_strict_consistency_googlegemini = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/GoogleGemini/strict_consistency/'
path_context_strict_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/context/ChatGPT/strict_consistency/'
path_context_strict_consistency_googlegemini = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/context/GoogleGemini/strict_consistency/'

#soft consistency
path_no_context_soft_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/ChatGPT/soft_consistency/'
path_no_context_soft_consistency_googlebard = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/GoogleBard/soft_consistency/'
path_no_context_soft_consistency_googlegemini = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/no-context/GoogleGemini/soft_consistency/'
path_context_soft_consistency_chatgpt = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/context/ChatGPT/soft_consistency/'
path_context_soft_consistency_googlegemini = '/Users/brain/Documents/GitHub/LLMs-topic-classification/results/mapped_csv/context/GoogleGemini/soft_consistency/'

dataset_paths_list_strict = [path_no_context_strict_consistency_chatgpt,
                             path_no_context_strict_consistency_googlebard,
                             path_no_context_strict_consistency_googlegemini,
                             path_context_strict_consistency_chatgpt,
                             path_context_strict_consistency_googlegemini]
                      
dataset_paths_list_soft = [path_no_context_soft_consistency_chatgpt,
                           path_no_context_soft_consistency_googlebard,
                           path_no_context_soft_consistency_googlegemini,
                           path_context_soft_consistency_chatgpt,
                           path_context_soft_consistency_googlegemini]

dataset_paths_list_context = [path_context_strict_consistency_chatgpt,
                              path_context_strict_consistency_googlegemini,
                              path_context_soft_consistency_chatgpt,
                              path_context_soft_consistency_googlegemini]

dataset_paths_list_no_context = [path_no_context_strict_consistency_chatgpt,
                                 path_no_context_strict_consistency_googlebard,
                                 path_no_context_strict_consistency_googlegemini,
                                 path_no_context_soft_consistency_chatgpt,
                                 path_no_context_soft_consistency_googlebard,
                                 path_no_context_soft_consistency_googlegemini]

#datasets names list
dataset_names_list = ['Education_expenditure_and_indicators.csv',
                      'Health_expectancy.csv',
                      'Listed_monuments.csv', 
                      'Livestock.csv',
                      'Milk_supply_and_dairy_production.csv',
                      'Mobility.csv',
                      'Plant_protection_products.csv',
                      'Population_dynamics.csv',
                      'Social_security.csv',
                      'Trade_and_industry.csv']

In [45]:
row_pairs = [('run1', 'run1'), ('run1', 'run2'), ('run1', 'run3'), ('run1', 'run4'), ('run1', 'run5'), 
             ('run1', 'run6'), ('run1', 'run7'), ('run1', 'run8'), ('run1', 'run9'), ('run1', 'run10'), 
             ('run2', 'run1'), ('run2', 'run2'), ('run2', 'run3'), ('run2', 'run4'), ('run2', 'run5'), 
             ('run2', 'run6'), ('run2', 'run7'), ('run2', 'run8'), ('run2', 'run9'), ('run2', 'run10'), 
             ('run3', 'run1'), ('run3', 'run2'), ('run3', 'run3'), ('run3', 'run4'), ('run3', 'run5'), 
             ('run3', 'run6'), ('run3', 'run7'), ('run3', 'run8'), ('run3', 'run9'), ('run3', 'run10'), 
             ('run4', 'run1'), ('run4', 'run2'), ('run4', 'run3'), ('run4', 'run4'), ('run4', 'run5'),
             ('run4', 'run6'), ('run4', 'run7'), ('run4', 'run8'), ('run4', 'run9'), ('run4', 'run10'),
             ('run5', 'run1'), ('run5', 'run2'), ('run5', 'run3'), ('run5', 'run4'), ('run5', 'run5'),
             ('run5', 'run6'), ('run5', 'run7'), ('run5', 'run8'), ('run5', 'run9'), ('run5', 'run10'),
             ('run6', 'run1'), ('run6', 'run2'), ('run6', 'run3'), ('run6', 'run4'), ('run6', 'run5'),
             ('run6', 'run6'), ('run6', 'run7'), ('run6', 'run8'), ('run6', 'run9'), ('run6', 'run10'), 
             ('run7', 'run1'), ('run7', 'run2'), ('run7', 'run3'), ('run7', 'run4'), ('run7', 'run5'), 
             ('run7', 'run6'), ('run7', 'run7'), ('run7', 'run8'), ('run7', 'run9'), ('run7', 'run10'), 
             ('run8', 'run1'), ('run8', 'run2'), ('run8', 'run3'), ('run8', 'run4'), ('run8', 'run5'), 
             ('run8', 'run6'), ('run8', 'run7'), ('run8', 'run8'), ('run8', 'run9'), ('run8', 'run10'), 
             ('run9', 'run1'), ('run9', 'run2'), ('run9', 'run3'), ('run9', 'run4'), ('run9', 'run5'), 
             ('run9', 'run6'), ('run9', 'run7'), ('run9', 'run8'), ('run9', 'run9'), ('run9', 'run10'),
             ('run10', 'run1'), ('run10', 'run2'), ('run10', 'run3'), ('run10', 'run4'), ('run10', 'run5'), 
             ('run10', 'run6'), ('run10', 'run7'), ('run10', 'run8'), ('run10', 'run9'), ('run10', 'run10')]

In [46]:
def remove_id_column(df):
    
    # Drop the column 'ID' if it exists
    if 'ID' in df.columns:
        df = df.drop(columns=['ID'])
    
    return df

In [47]:
def set_index_column(df):
    
    # Set run_index as index column
    df.set_index('run_index', inplace=True)
    
    return df

In [48]:
def transform_row_to_sequence(df, row_pair):
    
    # Extract row values to list
    seq1 = df.loc[row_pair[0]].tolist()
    seq2 = df.loc[row_pair[1]].tolist()
    
    return seq1, seq2

In [49]:
def remove_not_found_from_sequence(seq):
    
    seq = [x for x in seq if x != 'NOT_FOUND']
    
    return seq

In [113]:
"""
def get_alignment_score(seq1, seq2, n_cols1, n_cols2):
    
    alignment: needle.NeedlemanWunsch[str] = needle.NeedlemanWunsch(seq1, seq2)
    
    alignment.change_matrix(core.ScoreMatrix(match=1, miss=-0.5, gap=-2))
    
    if len(seq1) == 0 and len(seq2) == 0:
        pair_alignment_score = -2
        return pair_alignment_score
    
    else:
        alignment.align()
        
        if len(seq1) > len(seq2):
            longer_seq = seq1
        elif len(seq1) < len(seq2):
            longer_seq = seq2
        else:
            longer_seq = seq1  # or seq2, since they are the same length
        
        if n_cols1 > n_cols2:
            max_n_cols = n_cols1
        elif n_cols1 < n_cols2:
            max_n_cols = n_cols2
        else:
            max_n_cols = n_cols1
            
        if max_n_cols > len(longer_seq):
            additional_penalty = -2*(max_n_cols-len(longer_seq)) # additional penalty for when both pairs have missing values
            pair_alignment_score = (alignment.get_score()+(additional_penalty))/max_n_cols #dividing by the number of columns
            return pair_alignment_score
        else:
            pair_alignment_score = alignment.get_score()/max_n_cols #dividing by the number of columns
            return pair_alignment_score
"""

'\ndef get_alignment_score(seq1, seq2, n_cols1, n_cols2):\n    \n    alignment: needle.NeedlemanWunsch[str] = needle.NeedlemanWunsch(seq1, seq2)\n    \n    alignment.change_matrix(core.ScoreMatrix(match=1, miss=-0.5, gap=-2))\n    \n    if len(seq1) == 0 and len(seq2) == 0:\n        pair_alignment_score = -2\n        return pair_alignment_score\n    \n    else:\n        alignment.align()\n        \n        if len(seq1) > len(seq2):\n            longer_seq = seq1\n        elif len(seq1) < len(seq2):\n            longer_seq = seq2\n        else:\n            longer_seq = seq1  # or seq2, since they are the same length\n        \n        if n_cols1 > n_cols2:\n            max_n_cols = n_cols1\n        elif n_cols1 < n_cols2:\n            max_n_cols = n_cols2\n        else:\n            max_n_cols = n_cols1\n            \n        if max_n_cols > len(longer_seq):\n            additional_penalty = -2*(max_n_cols-len(longer_seq)) # additional penalty for when both pairs have missing values\n 

In [ ]:
def get_alignment_score(seq1, seq2, n_cols1, n_cols2):
    
    alignment: needle.NeedlemanWunsch[str] = needle.NeedlemanWunsch(seq1, seq2)
    
    alignment.change_matrix(core.ScoreMatrix(match=1, miss=-0.5, gap=-2))
    
    if len(seq1) == 0 and len(seq2) == 0:
        pair_alignment_score = -2
        return pair_alignment_score
    
    else:
        alignment.align()
        
        if len(seq1) > len(seq2):
            longer_seq = seq1
        elif len(seq1) < len(seq2):
            longer_seq = seq2
        else:
            longer_seq = seq1  # or seq2, since they are the same length
        
        if n_cols1 > n_cols2:
            max_n_cols = n_cols1
        elif n_cols1 < n_cols2:
            max_n_cols = n_cols2
        else:
            max_n_cols = n_cols1
            
        if max_n_cols > len(longer_seq):
            additional_penalty = -2*(max_n_cols-len(longer_seq)) # additional penalty for when both pairs have missing values
            pair_alignment_score = (alignment.get_score()+(additional_penalty))/max_n_cols #dividing by the number of columns
            return pair_alignment_score
        else:
            pair_alignment_score = alignment.get_score()/max_n_cols #dividing by the number of columns
            return pair_alignment_score

In [51]:
def calculate_alignment_datasets_pair(row_pairs, df1, df2):
    
    for row_pair in row_pairs:
        
        seq1 = df1.loc[row_pair[0]].tolist()
        seq2 = df2.loc[row_pair[1]].tolist()
        
        seq1 = remove_not_found_from_sequence(seq1)
        seq2 = remove_not_found_from_sequence(seq2)
        
        n_cols_df1 = df1.shape[1]
        n_cols_df2 = df2.shape[1]
        
        dataset_alignment_score = get_alignment_score(seq1, seq2, n_cols_df1, n_cols_df2)
        
        return dataset_alignment_score
    

In [52]:
# ONLY STRICT CONSISTENCY + NO-CONTEXT

In [53]:
results_df_strict_no_context = pd.DataFrame(columns=['setting', 'system_pair', 'dataset', 'alignment_score'])

In [54]:
for dataset_name in dataset_names_list:
    df_no_context_strict_chatgpt = pd.read_csv(os.path.join(path_no_context_strict_consistency_chatgpt, dataset_name))
    df_no_context_strict_googlebard = pd.read_csv(os.path.join(path_no_context_strict_consistency_googlebard, dataset_name))
    df_no_context_strict_googlegemini = pd.read_csv(os.path.join(path_no_context_strict_consistency_googlegemini, dataset_name))
    
    df_no_context_strict_chatgpt = remove_id_column(df_no_context_strict_chatgpt)
    df_no_context_strict_googlebard = remove_id_column(df_no_context_strict_googlebard)
    df_no_context_strict_googlegemini = remove_id_column(df_no_context_strict_googlegemini)
    
    df_no_context_strict_chatgpt = set_index_column(df_no_context_strict_chatgpt)
    df_no_context_strict_googlebard = set_index_column(df_no_context_strict_googlebard)
    df_no_context_strict_googlegemini = set_index_column(df_no_context_strict_googlegemini)
    
    # chatgpt vs bard
    
    dataset_alignment_score_no_context_strict_gpt_bard = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_no_context_strict_chatgpt,
                                                                                          df_no_context_strict_googlebard)
    
    results_df_strict_no_context.loc[len(results_df_strict_no_context)] = ["no-context/strict", 
                                                                           "ChatGPTvsGoogleBard" ,
                                                                            dataset_name, 
                                                    dataset_alignment_score_no_context_strict_gpt_bard/len(row_pairs)]
    
    # chatgpt vs gemini
    dataset_alignment_score_no_context_strict_gpt_gemini = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_no_context_strict_chatgpt,
                                                                                          df_no_context_strict_googlegemini)
    
    results_df_strict_no_context.loc[len(results_df_strict_no_context)] = ["no-context/strict", 
                                                                           "ChatGPTvsGoogleGemini" ,
                                                                           dataset_name, 
                                                     dataset_alignment_score_no_context_strict_gpt_gemini/len(row_pairs)]
    
    # bard vs gemini
    dataset_alignment_score_no_context_strict_bard_gemini = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_no_context_strict_googlebard,
                                                                                          df_no_context_strict_googlegemini)
    
    results_df_strict_no_context.loc[len(results_df_strict_no_context)] = ["no-context/strict", 
                                                                           "GoogleBardvsGoogleGemini" ,
                                                                           dataset_name, 
                                                     dataset_alignment_score_no_context_strict_bard_gemini/len(row_pairs)]
    

In [55]:
# ONLY SOFT CONSISTENCY + NO-CONTEXT

In [56]:
results_df_soft_no_context = pd.DataFrame(columns=['setting', 'system_pair', 'dataset', 'alignment_score'])

In [57]:
for dataset_name in dataset_names_list:
    df_no_context_soft_chatgpt = pd.read_csv(os.path.join(path_no_context_soft_consistency_chatgpt, dataset_name))
    df_no_context_soft_googlebard = pd.read_csv(os.path.join(path_no_context_soft_consistency_googlebard, dataset_name))
    df_no_context_soft_googlegemini = pd.read_csv(os.path.join(path_no_context_soft_consistency_googlegemini, dataset_name))
    
    df_no_context_soft_chatgpt = remove_id_column(df_no_context_soft_chatgpt)
    df_no_context_soft_googlebard = remove_id_column(df_no_context_soft_googlebard)
    df_no_context_soft_googlegemini = remove_id_column(df_no_context_soft_googlegemini)
    
    df_no_context_soft_chatgpt = set_index_column(df_no_context_soft_chatgpt)
    df_no_context_soft_googlebard = set_index_column(df_no_context_soft_googlebard)
    df_no_context_soft_googlegemini = set_index_column(df_no_context_soft_googlegemini)
    
    # chatgpt vs bard
    
    dataset_alignment_score_no_context_soft_gpt_bard = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_no_context_soft_chatgpt,
                                                                                          df_no_context_soft_googlebard)
    
    results_df_soft_no_context.loc[len(results_df_soft_no_context)] = ["no-context/soft", 
                                                                           "ChatGPTvsGoogleBard" ,
                                                                            dataset_name, 
                                                    dataset_alignment_score_no_context_soft_gpt_bard/len(row_pairs)]
    
    # chatgpt vs gemini
    dataset_alignment_score_no_context_soft_gpt_gemini = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_no_context_soft_chatgpt,
                                                                                          df_no_context_soft_googlegemini)
    
    results_df_soft_no_context.loc[len(results_df_soft_no_context)] = ["no-context/soft", 
                                                                           "ChatGPTvsGoogleGemini" ,
                                                                           dataset_name, 
                                                     dataset_alignment_score_no_context_soft_gpt_gemini/len(row_pairs)]
    
    # bard vs gemini
    dataset_alignment_score_no_context_soft_bard_gemini = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_no_context_soft_googlebard,
                                                                                          df_no_context_soft_googlegemini)
    
    results_df_soft_no_context.loc[len(results_df_soft_no_context)] = ["no-context/soft", 
                                                                           "GoogleBardvsGoogleGemini" ,
                                                                           dataset_name, 
                                                     dataset_alignment_score_no_context_soft_bard_gemini/len(row_pairs)]
    

In [58]:
# ONLY STRICT CONSISTENCY + CONTEXT

In [59]:
results_df_strict_context = pd.DataFrame(columns=['setting', 'system_pair', 'dataset', 'alignment_score'])

In [60]:
for dataset_name in dataset_names_list:
    df_context_strict_chatgpt = pd.read_csv(os.path.join(path_context_strict_consistency_chatgpt, dataset_name))
    df_context_strict_googlegemini = pd.read_csv(os.path.join(path_context_strict_consistency_googlegemini, dataset_name))
    
    df_context_strict_chatgpt = remove_id_column(df_context_strict_chatgpt)
    df_context_strict_googlegemini = remove_id_column(df_context_strict_googlegemini)
    
    df_context_strict_chatgpt = set_index_column(df_context_strict_chatgpt)
    df_context_strict_googlegemini = set_index_column(df_context_strict_googlegemini)
    
    
    # chatgpt vs gemini
    dataset_alignment_score_context_strict_gpt_gemini = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_context_strict_chatgpt,
                                                                                          df_context_strict_googlegemini)
    
    results_df_strict_context.loc[len(results_df_strict_context)] = ["context/strict", 
                                                                           "ChatGPTvsGoogleGemini" ,
                                                                           dataset_name, 
                                                     dataset_alignment_score_context_strict_gpt_gemini/len(row_pairs)]
    

In [61]:
# ONLY SOFT CONSISTENCY + CONTEXT

In [62]:
results_df_soft_context = pd.DataFrame(columns=['setting', 'system_pair', 'dataset', 'alignment_score'])

In [63]:
for dataset_name in dataset_names_list:
    df_context_soft_chatgpt = pd.read_csv(os.path.join(path_context_soft_consistency_chatgpt, dataset_name))
    df_context_soft_googlegemini = pd.read_csv(os.path.join(path_context_soft_consistency_googlegemini, dataset_name))
    
    df_context_soft_chatgpt = remove_id_column(df_context_soft_chatgpt)
    df_context_soft_googlegemini = remove_id_column(df_context_soft_googlegemini)
    
    df_context_soft_chatgpt = set_index_column(df_context_soft_chatgpt)
    df_context_soft_googlegemini = set_index_column(df_context_soft_googlegemini)
    
    
    # chatgpt vs gemini
    dataset_alignment_score_context_soft_gpt_gemini = calculate_alignment_datasets_pair(row_pairs,
                                                                                          df_context_soft_chatgpt,
                                                                                          df_context_soft_googlegemini)
    
    results_df_soft_context.loc[len(results_df_soft_context)] = ["context/soft", 
                                                                           "ChatGPTvsGoogleGemini" ,
                                                                           dataset_name, 
                                                     dataset_alignment_score_context_soft_gpt_gemini/len(row_pairs)]
    

In [64]:
combined_df = pd.concat([results_df_strict_no_context, 
                         results_df_soft_no_context,
                         results_df_strict_context,
                         results_df_soft_context],
                         ignore_index=True)

In [ ]:
combined_df

In [ ]:
combined_df.to_csv('/Users/brain/Documents/GitHub/LLMs-topic-classification/results/statistics/NeedlemanWunsch_intra_llm_score.csv')

In [75]:
combined_df

,setting,system_pair,dataset,alignment_score
0,no-context/strict,ChatGPTvsGoogleBard,Education_expenditure_and_indicators.csv,-0.004434
1,no-context/strict,ChatGPTvsGoogleGemini,Education_expenditure_and_indicators.csv,-0.004434
2,no-context/strict,GoogleBardvsGoogleGemini,Education_expenditure_and_indicators.csv,-0.003824
3,no-context/strict,ChatGPTvsGoogleBard,Health_expectancy.csv,-0.001786
4,no-context/strict,ChatGPTvsGoogleGemini,Health_expectancy.csv,0.002500
...,...,...,...,...
75,context/soft,ChatGPTvsGoogleGemini,Mobility.csv,0.000000
76,context/soft,ChatGPTvsGoogleGemini,Plant_protection_products.csv,-0.005000
77,context/soft,ChatGPTvsGoogleGemini,Population_dynamics.csv,0.005000
78,context/soft,ChatGPTvsGoogleGemini,Social_security.csv,0.007632


In [ ]:
# CHATGPT vs BARD (no-context)

In [76]:
df_gpt_bard = combined_df[combined_df['system_pair'] == 'ChatGPTvsGoogleBard']

In [79]:
model = ols('alignment_score ~ setting + dataset', data=df_gpt_bard).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
setting,0.000001,1.0,3.662812,8.791849e-02
dataset,0.000468,9.0,148.926006,1.050164e-08
Residual,0.000003,9.0,NaN,NaN


In [124]:
tukey_results_system = pairwise_tukeyhsd(df_gpt_gemini['alignment_score'], df_gpt_gemini['dataset'])
print(tukey_results_system)

                                Multiple Comparison of Means - Tukey HSD, FWER=0.05                                
                 group1                                 group2                meandiff p-adj   lower  upper  reject
-------------------------------------------------------------------------------------------------------------------
Education_expenditure_and_indicators.csv                Health_expectancy.csv   0.0058 0.5779 -0.0039 0.0156  False
Education_expenditure_and_indicators.csv                 Listed_monuments.csv  -0.0006    1.0 -0.0103 0.0091  False
Education_expenditure_and_indicators.csv                        Livestock.csv   0.0007    1.0 -0.0091 0.0104  False
Education_expenditure_and_indicators.csv Milk_supply_and_dairy_production.csv  -0.0009    1.0 -0.0107 0.0088  False
Education_expenditure_and_indicators.csv                         Mobility.csv   0.0023 0.9978 -0.0074 0.0121  False
Education_expenditure_and_indicators.csv        Plant_protection_product

In [114]:
# CHATGPT vs GEMINI (no-context)

In [125]:
df_gpt_gemini_no_context = combined_df[combined_df['system_pair'] == 'GoogleBardvsGoogleGemini']
df_gpt_gemini_no_context = df_gpt_gemini_no_context[df_gpt_gemini_no_context['setting'].isin(['no-context/strict', 'no-context/soft'])]

In [122]:
model = ols('alignment_score ~ setting + dataset', data=df_gpt_gemini_no_context).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
setting,0.000037,1.0,4.054561,0.074892
dataset,0.000129,9.0,1.573870,0.254953
Residual,0.000082,9.0,NaN,NaN


In [ ]:
# Bard vs GEMINI (no-context)

In [126]:
df_bard_gemini_no_context = combined_df[combined_df['system_pair'] == 'GoogleBardvsGoogleGemini']

In [127]:
model = ols('alignment_score ~ setting + dataset', data=df_bard_gemini_no_context).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
setting,0.000002,1.0,3.475530,9.516270e-02
dataset,0.000800,9.0,201.096071,2.753239e-09
Residual,0.000004,9.0,NaN,NaN


In [83]:
# CHATGPT vs GEMINI

In [100]:
df_gpt_gemini = combined_df[combined_df['system_pair'] == 'ChatGPTvsGoogleGemini']

In [101]:
df_gpt_gemini[['context', 'softness']] = df_gpt_gemini['setting'].str.split('/', expand=True)
df_gpt_gemini = df_gpt_gemini.drop(columns=['setting'])
df_gpt_gemini = df_gpt_gemini.drop(columns=['system_pair'])

/var/folders/ts/b1943r396fj_jxc57wkd2y040000gn/T/ipykernel_35701/2498313190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gpt_gemini[['context', 'softness']] = df_gpt_gemini['setting'].str.split('/', expand=True)
/var/folders/ts/b1943r396fj_jxc57wkd2y040000gn/T/ipykernel_35701/2498313190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gpt_gemini[['context', 'softness']] = df_gpt_gemini['setting'].str.split('/', expand=True)


In [104]:
model = ols('alignment_score ~ dataset + context + softness', data=df_gpt_gemini).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
dataset,0.000252,9.0,2.003732,0.077177
context,0.000020,1.0,1.454914,0.237835
softness,0.000077,1.0,5.472781,0.026676
Residual,0.000392,28.0,NaN,NaN


In [106]:
tukey_results_system = pairwise_tukeyhsd(df_gpt_gemini['alignment_score'], df_gpt_gemini['softness'])
print(tukey_results_system)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
  soft strict  -0.0028 0.0431 -0.0054 -0.0001   True
----------------------------------------------------


In [107]:
# BARD vs GEMINI

In [108]:
df_bard_gemini = combined_df[combined_df['system_pair'] == 'GoogleBardvsGoogleGemini']

In [110]:
model = ols('alignment_score ~ setting + dataset', data=df_bard_gemini).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
setting,0.000002,1.0,3.475530,9.516270e-02
dataset,0.000800,9.0,201.096071,2.753239e-09
Residual,0.000004,9.0,NaN,NaN


In [112]:
2.753239e-09 < 0.05

True